Importing Packages 

In [1]:
import pandas as pd
from pathlib import Path
import time
import tqdm
from datetime import datetime
import random
import os
from sklearn.model_selection import KFold
import sys
import pytorch_lightning as plit
import pandas as pd
import math
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler, normalize, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_recall_curve, confusion_matrix, roc_curve, matthews_corrcoef
from sklearn.metrics import auc as skauc
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
import numpy as np
from multiprocessing import Pool
import multiprocessing
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from xgboost import XGBClassifier, XGBRegressor
import torch.nn as nn
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader, TensorDataset
import torch
import argparse
import dateutil
import shap
import re
import torch.nn.functional as F
from fancyimpute import IterativeImputer as MICE

# K-fold validation
n_splits = 5  
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

Loading Prescriptions and Admissions File

In [2]:
#Load Prescriptions CSV File into Pandas Dataframe 
prescriptions_df = pd.read_csv("dataset/PRESCRIPTIONS.csv")
#Load Admissions CSV File into  Dataframe 
admissions_df = pd.read_csv("dataset/ADMISSIONS.csv",low_memory=False)

Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.


In [3]:
# Selected type (dose or count)
selected_type = 'dose'

# Define conversion factors for dosage units
conversion_factors = {
    'mg': 1,
    'mcg': 0.001,
    'g': 1000,
    'pill': 50,
    'tab': 50,
    'gm': 1000,
    'mEq': 74.5,
    'mL': 1,
    'UNIT': 100,
    'mcg/hr': 0.001,
    'mg/hr': 1,
}

# Convert DOSE_UNIT_RX to conversion factors and multiply by DOSE_VAL_RX to get dose in mg
prescriptions_df['DOSE_VAL_RX'] = pd.to_numeric(prescriptions_df['DOSE_VAL_RX'], errors='coerce').fillna(0)
prescriptions_df['dose_mg'] = prescriptions_df['DOSE_VAL_RX'] * prescriptions_df['DOSE_UNIT_RX'].map(conversion_factors).fillna(1)

# Group by unique patient ID and DRUG to count occurrences and sum dosages
grouped_counts = prescriptions_df.groupby(['SUBJECT_ID', 'DRUG']).size().unstack(fill_value=0)
grouped_dosages = prescriptions_df.groupby(['SUBJECT_ID', 'DRUG'])['dose_mg'].sum().unstack(fill_value=0)

if selected_type == 'count':
    # Concatenate the counts and dosage DataFrames, ensuring we keep the SUBJECT_ID as a column
    final_df = grouped_counts 
elif selected_type == 'dose':
    final_df = grouped_dosages
 
# Reset the index to convert 'SUBJECT_ID' back to a regular column
final_df.reset_index(inplace=True)

# Add a column for the number of unique drugs taken per patient
final_df['num_unique_drugs'] = (grouped_counts > 0).sum(axis=1)

# Add a column indicating if any drugs were taken by a patient (1 if any drug taken, 0 otherwise)
final_df['any_drug_taken'] = final_df['num_unique_drugs'].apply(lambda x: 1 if x > 0 else 0)


# Autoencoder for prescriptions

In [15]:
from collections import defaultdict

class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE, self).__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        
        # Encoder layers
        self.encoder_fc1 = nn.Linear(input_dim, 256)
        self.encoder_bn1 = nn.BatchNorm1d(256)
        self.encoder_fc2_mean = nn.Linear(256, latent_dim)
        self.encoder_fc2_logvar = nn.Linear(256, latent_dim)
        
        # Decoder layers
        self.decoder_fc1 = nn.Linear(latent_dim, 256)
        self.decoder_bn1 = nn.BatchNorm1d(256)
        self.decoder_fc2 = nn.Linear(256, input_dim)

        # Initialize weights
        self._initialize_weights()
        
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def encode(self, x):
        h1 = F.relu(self.encoder_bn1(self.encoder_fc1(x)))
        return self.encoder_fc2_mean(h1), self.encoder_fc2_logvar(h1)

    def decode(self, z):
        h3 = F.relu(self.decoder_bn1(self.decoder_fc1(z)))
        return torch.sigmoid(self.decoder_fc2(h3))

    def forward(self, x):
        mu, log_var = self.encode(x.view(-1, self.input_dim))
        z = self.reparameterize(mu, log_var)
        return self.decode(z), mu, log_var
    
    def infer(self, x):
        # Encoding
        z_mean, _ = self.encode(x)
        z_mean = z_mean.detach().numpy()[0]
        return z_mean
        
class AutoEncoder:
    def __init__(self, prescriptions,
                 max_length = 100,
                 latent_dim = 3,
                 threshold = 1):
        
        subject_codes = defaultdict(list)
        self.code_converter = {}
        for num, row in tqdm.tqdm(prescriptions.iterrows(), total = len(prescriptions), desc = 'Creating code features'):
            subject_id = row['SUBJECT_ID']
            diagnoses_list = row['drugs_taken']
            for diagnoses in diagnoses_list:
                try:
                    subject_codes[subject_id].append(self.code_converter[diagnoses])
                except:
                    self.code_converter[diagnoses] = len(self.code_converter)
                    subject_codes[subject_id].append(self.code_converter[diagnoses])
                
        subject_codes = dict(subject_codes)
        self.subjects = list(subject_codes.keys())

        # Obtains tensors
        self.input_tensors = []
        self.max_length = max_length
        for num, (subject_id, codes) in tqdm.tqdm(enumerate(subject_codes.items()), total = len(subject_codes), desc = 'Padding samples'):
            while len(codes) < max_length:
                codes.append(0)
            if len(codes) > max_length:
                codes = random.sample(codes, max_length)
            self.input_tensors.append(torch.FloatTensor(codes))

        # Scales data
        self.scaler = StandardScaler()
        self.input_tensors = torch.FloatTensor(self.scaler.fit_transform(self.input_tensors))
        
        # Makes VAE
        self.autoencoder = VAE(input_dim = self.max_length, latent_dim = latent_dim)

    def train_VAE(self, batch_size = 32,
                 epochs = 10):
        
        # Define the DataLoader
        batch_size = 16
        data_loader = DataLoader(self.input_tensors, batch_size=batch_size, shuffle=True)

        # Define the loss function and optimizer
        def loss_function(reconstructed_x, x, mu, log_var):
            # Reconstruction loss (MSE)
            recon_loss = F.mse_loss(reconstructed_x, x) / x.size(0)  # Divide by batch size for stability
            
            # KL divergence
            kld_loss = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp()) / x.size(0)
        
            return recon_loss + kld_loss

        # Modify the optimizer to AdamW for improved weight regularization
        optimizer = torch.optim.AdamW(self.autoencoder.parameters(), lr=0.001)
        #scheduler = StepLR(optimizer, step_size=10, gamma=0.5)  # Adjust learning rate

        # Training loop with early stopping based on validation loss
        best_loss = float('inf')
        patience, trials = 5, 0  # Patience: How long to wait after last time validation loss improved.
                                 # Trials: How many times we allowed the validation loss to not improve.
        for epoch in range(epochs):
            total_loss = 0
            for data in tqdm.tqdm(data_loader, desc=f'Epoch {epoch}'):
                optimizer.zero_grad()
                reconstruction, mu, log_var = self.autoencoder(data)
                loss = loss_function(reconstruction, data, mu, log_var)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            avg_loss = total_loss / len(data_loader)
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}')
            #scheduler.step()

            # Early stopping logic
            if avg_loss < best_loss:
                best_loss = avg_loss
                trials = 0
            else:
                trials += 1
                if trials >= patience:
                    print("Early stopping triggered.")
                    break
                    
        self.autoencoder.eval()
        
        encoded_features = []
        for tensor in tqdm.tqdm(self.input_tensors, total=len(self.input_tensors), desc='Encoding...'):
            # Ensure tensor is in the right shape for the model
            tensor = tensor.unsqueeze(0)  # Adds a batch dimension
            encoded_feature = self.autoencoder.infer(tensor)
            encoded_features.append(encoded_feature)
        
        # Convert the list of numpy arrays to a single numpy array
        encoded_features = np.array(encoded_features)

    def extract_features(self, data, scale = True):
        data = np.array(data)
        self.autoencoder.eval()
        
        if len(data) < self.max_length:    
            while len(data) < self.max_length:
                data = np.append(data, 0)
                
        elif len(data) > self.max_length:
            data = data[self.max_length]

        data = self.scaler.transform([data])
        
        outputs = self.autoencoder.infer(torch.FloatTensor(data)).detach().numpy()[0]
        return outputs

# Create a dictionary to hold the list of drugs for each SUBJECT_ID
subject_drugs = {}

# Iterate over the rows of the DataFrame to populate the dictionary
for index, row in tqdm.tqdm(prescriptions_df.iterrows(), total = len(prescriptions_df)):
    subject_id = row['SUBJECT_ID']
    drug = row['DRUG']
    # Initialize the subject's drug list if not already present
    if subject_id not in subject_drugs:
        subject_drugs[subject_id] = []
    # Add the drug to the list for the subject if not already included
    if drug not in subject_drugs[subject_id]:
        subject_drugs[subject_id].append(drug)

max_length = len(sorted(subject_drugs.items(), key = lambda x: len(x[1]), reverse = True)[0][1])
# Convert the dictionary to a DataFrame
drugs_df = pd.DataFrame(list(subject_drugs.items()), columns=['SUBJECT_ID', 'drugs_taken'])

# Merge this DataFrame with the final_df on SUBJECT_ID
token_df = pd.merge(final_df, drugs_df, on='SUBJECT_ID', how='left')

VAE = AutoEncoder(prescriptions = token_df, max_length = max_length)
features = pd.DataFrame(np.array(VAE.train_VAE(epochs = 2)))
print(features.shape)

# Tests models
test_models(features, 
                mortality_labels['mortality_label'], # Mortality labels
                feature_names = None, # Feature names 
                equalize = True,
                rescale = True) # Equalize labels  

Epoch 0: 100%|██████████| 2461/2461 [12:38<00:00,  3.24it/s]


Epoch [1/2], Loss: 0.1622


Epoch 1: 100%|██████████| 2461/2461 [12:17<00:00,  3.34it/s]


Epoch [2/2], Loss: 0.0648


Encoding...:   0%|          | 6/39363 [00:00<11:03, 59.33it/s]

...

Encoding...:  97%|█████████▋| 38335/39363 [01:41<00:00, 2968.83it/s]

torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1

Encoding...: 100%|██████████| 39363/39363 [01:41<00:00, 388.11it/s] 


torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1, 220])
torch.Size([1

ValueError: Must pass 2-d input. shape=()

In [10]:
# Filters out labels within a given timeframe
def process_data_with_timeframe_filtering(admit_times, discharge_times, death_times, filter_within_timeframe, max_hours, subject_ids):
    death_labels = []
    hospital_stay = []
    filtered_ids = []
    
    for admit, dis, death, subject_id in tqdm.tqdm(zip(admit_times, discharge_times, death_times, subject_ids), total=len(admit_times), desc='Processing mortality'):
        within_timeframe = False
        death_time = (death - admit).total_seconds() if pd.notnull(death) else None
        discharge_time = (dis - admit).total_seconds() if pd.notnull(dis) else None
        
        if death_time and death_time <= max_hours * 3600:
            within_timeframe = True
        elif discharge_time and discharge_time <= max_hours * 3600:
            within_timeframe = True
        
        if filter_within_timeframe and within_timeframe:
            continue  # Skip this patient
        
        # Add the patient's data
        hospital_stay.append(death_time if death_time else discharge_time)
        death_labels.append(1 if death_time else 0)
        filtered_ids.append(subject_id)
    
    return death_labels, hospital_stay, filtered_ids

#ADMISSIONS FILE 
admissions_df['ADMITTIME'] = pd.to_datetime(admissions_df['ADMITTIME'],)
admissions_df['DISCHTIME'] = pd.to_datetime(admissions_df['DISCHTIME'])
admissions_df['DEATHTIME'] = pd.to_datetime(admissions_df['DEATHTIME'],)
    
# New logic to filter patients based on discharge/death time within the timeframe
admit_times = admissions_df['ADMITTIME']
mapping_discharge = {data['SUBJECT_ID']: data['ADMITTIME'] for _, data in admissions_df.iterrows()}
discharge_times = admissions_df['DISCHTIME']
death_times = admissions_df['DEATHTIME']

# Filter based on discharge/death time within the timeframe and other logic...
death_labels, hospital_stay, filtered_ids = process_data_with_timeframe_filtering(
    admit_times, discharge_times, death_times, False, 0, subject_ids = admissions_df['SUBJECT_ID'],)

mortality_labels = pd.DataFrame({
    'SUBJECT_ID': filtered_ids,
    'mortality_label':death_labels
})

merged_df = final_df.merge(mortality_labels, on = 'SUBJECT_ID', how = 'left')
    
# Check for duplicate subject IDs in final_df
duplicate_subject_ids_final = final_df[final_df.duplicated('SUBJECT_ID', keep=False)]

# Check for duplicate subject IDs in merged_df
duplicate_subject_ids_merged = merged_df[merged_df.duplicated('SUBJECT_ID', keep=False)]

# Print the lengths of the original dataframes and the merged dataframe
print("Length of admissions_df:", len(admissions_df))
print("Length of final_df:", len(final_df))
print("Length of merged_df:", len(merged_df))

merged_df = merged_df[merged_df['SUBJECT_ID'].isin(admissions_df['SUBJECT_ID'])]
merged_df.drop_duplicates(subset='SUBJECT_ID', keep=False, inplace=True)


Processing mortality: 100%|██████████| 58976/58976 [00:00<00:00, 137360.93it/s]


KeyboardInterrupt: 

In [ ]:
def analyze_features(features_scaled, feature_names, y, top_n_features=10, return_top_features=False):
    """
    Analyze features using SHAP values, Linear Regression, and PCA,
    select the top features from each method without duplicates, and visualize the results.

    :param features_scaled: Scaled feature numpy array
    :param feature_names: List of feature names
    :param y: True labels
    :param top_n_features: Number of top features to retain from each method
    """
    # Trains XGBoost classifier for SHAP
    xgb_model = XGBClassifier()
    xgb_model.fit(features_scaled, y)
    
    # Linear Regression for feature importance
    lr_model = LinearRegression()
    lr_model.fit(features_scaled, y)

    # SHAP values
    explainer_shap = shap.Explainer(xgb_model)
    shap_values = explainer_shap(features_scaled)
    sampled_features = shap.sample(features_scaled, 1000)
    explainer_tree = shap.TreeExplainer(xgb_model)
    shap_values_tree = explainer_tree.shap_values(sampled_features)

    # Feature importance from SHAP
    feature_importance_shap = np.abs(shap_values.values).mean(axis=0)
    top_features_shap = np.argsort(feature_importance_shap)[-top_n_features:]

    # Principal Component Analysis
    pca = PCA(n_components=len(feature_names))
    pca.fit(features_scaled)
    feature_importance_pca = pca.explained_variance_ratio_
    top_features_pca = np.argsort(feature_importance_pca)[-top_n_features:]

    # Visualization
    
    # Visualize SHAP values using summary plot
    shap.summary_plot(shap_values_tree, sampled_features, feature_names=feature_names)

    # Print feature importance based on SHAP
    feature_importance_shap_tree = np.abs(shap_values_tree).mean(axis=0)
    feature_importance_shap_tree = pd.Series(feature_importance_shap_tree, index=feature_names).sort_values(ascending=False)
    print("\nFeature importance based on SHAP TreeExplainer:")
    print(feature_importance_shap_tree.head(top_n_features))
    print("\n" + "-"*50 + "\n")

    # Linear Regression
    plt.figure(figsize=(10, 6))
    indices_lr = np.argsort(np.abs(lr_model.coef_))[::-1][:top_n_features]
    plt.title("Top Linear Regression Coefficients")
    plt.barh(range(top_n_features), np.abs(lr_model.coef_)[indices_lr], color='b', align='center')
    plt.yticks(range(top_n_features), [feature_names[i] for i in indices_lr])
    plt.xscale('log')
    plt.gca().invert_yaxis()  # Invert y-axis to have largest values at the top

    # PCA
    plt.figure(figsize=(10, 6))
    plt.title("Top PCA Explained Variance Ratio")
    plt.barh(range(top_n_features), feature_importance_pca[top_features_pca], color='b', align='center')
    plt.yticks(range(top_n_features), [feature_names[i] for i in top_features_pca])
    
    # Combine and remove duplicates
    combined_feature_indices = np.unique(np.concatenate((top_features_shap, indices_lr, top_features_pca)))
    combined_feature_names = [feature_names[i] for i in combined_feature_indices]

    print(f"Number of selected features: {len(combined_feature_names)}")
    print("Selected features:", combined_feature_names)

    if return_top_features:
        return combined_feature_names


def process_data_chunk_wrapper(chunk, id_conversion):
    """
    Calls process_data_chunk with the given chunk and id_conversion.
    """
    return process_data_chunk(chunk, id_conversion)
    
# Filters features based on their presence in the final patient dataset
def filter_features_on_presence(feature_df, feature_threshold =0.5, patient_threshold = 0.5):
    presence = feature_df.notna().mean()
    filtered_columns = presence[presence > feature_threshold].index
    feature_df = feature_df[filtered_columns]
    
    # Calculate the minimum number of non-NaN/None values required per row based on the proportion
    min_non_nan_count = int(np.ceil(patient_threshold * feature_df.shape[1]))
    
    # Filter rows based on the calculated minimum count of non-NaN/None values
    filtered_df = feature_df.dropna(axis=0, thresh=min_non_nan_count)
    
    return filtered_df
    
def filter_chart_data_optimized(chart_data, admissions_data, max_hours=48, chunks=4):
    # Ensure datetime format
    chart_data['CHARTTIME'] = pd.to_datetime(chart_data['CHARTTIME'], errors='coerce')
    admissions_data['ADMITTIME'] = pd.to_datetime(admissions_data['ADMITTIME'], errors='coerce')

    # Determine chunk size
    chunk_size = len(chart_data) // chunks
    filtered_data_list = []

    for i in range(chunks):
        # Calculate start and end index for each chunk
        start_idx = i * chunk_size
        end_idx = (i + 1) * chunk_size if i < chunks - 1 else len(chart_data)
        chart_data_chunk = chart_data.iloc[start_idx:end_idx]

        # Merge with admissions_data
        merged_data = chart_data_chunk.merge(admissions_data[['SUBJECT_ID', 'ADMITTIME']],
                                             on='SUBJECT_ID', how='inner')
        # Calculate time difference in hours
        merged_data['time_diff'] = (merged_data['CHARTTIME'] - merged_data['ADMITTIME']).dt.total_seconds() / 3600

        # Filter based on time_diff
        filtered_chunk = merged_data.query('0 <= time_diff <= @max_hours').drop(columns=['ADMITTIME', 'time_diff'])
        filtered_data_list.append(filtered_chunk)

    # Concatenate all filtered chunks
    filtered_data = pd.concat(filtered_data_list, ignore_index=True)
    
    return filtered_data
    
def extract_features_from_chart_data(chart_data, id_conversion, num_processes=16, min_non_nan_proportion=0.01):
    """
    Extract features from chart data using multiprocessing.
    """
    # Split data into chunks
    chunks = np.array_split(chart_data, num_processes)
    
    # Prepare arguments for each chunk processing
    args = [(chunk, id_conversion) for chunk in chunks]
    
    # Process each chunk in parallel
    with Pool(num_processes) as pool:
        results = pool.starmap(process_data_chunk_wrapper, args)
    
    # Combine results from all chunks
    combined = pd.concat(results, axis=0).reset_index(drop=True).groupby('SUBJECT_ID').first()
    
    # Determine the threshold number of non-NaN/non-None values required per column
    threshold_count = int(min_non_nan_proportion * len(combined))
    
    # Drop columns where the number of non-NaN/non-None values is less than the threshold
    filtered_combined = combined.dropna(axis=1, thresh=threshold_count, inplace=False)
    
    return filtered_combined 
    
def process_data_chunk(chunk, id_conversion, value_pick='last'):
    """
    Extract features from a chunk of data, preserving SUBJECT_ID.
    Args:
        - chunk (DataFrame): A chunk of the filtered chart data, preserving SUBJECT_ID.
        - id_conversion (dict): A dictionary for converting item IDs to feature names.
        - value_pick (str): Determines whether to pick the 'first' or 'last' non-empty value for each feature.
    Returns:
        - DataFrame with extracted features for the chunk, including SUBJECT_ID.
    """

    # Ensure SUBJECT_ID is considered during processing
    chunk['FEATURE_NAME'] = chunk['ITEMID'].map(id_conversion).fillna('Unknown')

    # Define aggregation function based on value_pick parameter
    if value_pick == 'first':
        value_func = lambda x: x.dropna().iloc[0] if not x.dropna().empty else None
    elif value_pick == 'last':
        value_func = lambda x: x.dropna().iloc[-1] if not x.dropna().empty else None
    elif value_pick == 'mean':
        value_func = lambda x: x.dropna().iloc[:] if not x.dropna().empty() else None
    else:
        raise ValueError("value_pick must be 'first' or 'last' or 'mean'")

    agg_funcs = {'VALUE': value_func}
    features = chunk.groupby(['SUBJECT_ID', 'FEATURE_NAME']).agg(agg_funcs).unstack()

    # Flatten Multi-Index columns
    features.columns = [f'{i}_{j}' for i, j in features.columns]
    return features.reset_index()
    
def filter_patients_within_timeframe(data, max_hours):

    max_seconds = max_hours * 3600
    # Mark patients to keep based on the condition
    keep_patients = []
    for i, row in data.iterrows():
        if pd.notnull(row['DEATHTIME']):
            time_diff = (row['DEATHTIME'] - row['ADMITTIME']).total_seconds()
        else:
            time_diff = (row['DISCHTIME'] - row['ADMITTIME']).total_seconds()
        
        if time_diff > max_seconds:
            keep_patients.append(row['SUBJECT_ID'])
    
    return keep_patients

# Filters out labels within a given timeframe
def process_data_with_timeframe_filtering(admit_times, discharge_times, death_times, filter_within_timeframe, max_hours, subject_ids):
    death_labels = []
    hospital_stay = []
    filtered_ids = []
    
    for admit, dis, death, subject_id in tqdm.tqdm(zip(admit_times, discharge_times, death_times, subject_ids), total=len(admit_times), desc='Processing mortality'):
        within_timeframe = False
        death_time = (death - admit).total_seconds() if pd.notnull(death) else None
        discharge_time = (dis - admit).total_seconds() if pd.notnull(dis) else None
        
        if death_time and death_time <= max_hours * 3600:
            within_timeframe = True
        elif discharge_time and discharge_time <= max_hours * 3600:
            within_timeframe = True
        
        if filter_within_timeframe and within_timeframe:
            continue  # Skip this patient
        
        # Add the patient's data
        hospital_stay.append(death_time if death_time else discharge_time)
        death_labels.append(1 if death_time else 0)
        filtered_ids.append(subject_id)
    
    return death_labels, hospital_stay, filtered_ids


In [ ]:
'''
# Temporarily remove "SUBJECT_ID" and "mortality_label" columns
subject_id = merged_df[['SUBJECT_ID']]
mortality_label = merged_df[['mortality_label']]
data_without_labels = merged_df.drop(columns=['SUBJECT_ID', 'mortality_label'])

# Continue with your processing logic, now without 'SUBJECT_ID' and 'mortality_label'
# Ensure these columns are not included in any of your feature lists
categorical_features = data_without_labels.select_dtypes(include=['object']).columns.tolist()
binary_features = [col for col in data_without_labels.columns if data_without_labels[col].nunique() == 2]
numeric_features = [col for col in data_without_labels.select_dtypes(include=['float64', 'int64']).columns if col not in categorical_features and col not in binary_features]
# Clean Numeric Features as before
for feature in numeric_features:
    data_without_labels[feature] = data_without_labels[feature].astype(str).str.replace(',', '').apply(lambda x: re.sub(r'(\d+)-(\d+)', r'\2', x))
    data_without_labels[feature] = pd.to_numeric(data_without_labels[feature], errors='coerce')

# Handle NaN values
data_without_labels.ffill(inplace=True)

# Apply transformations
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(drop='first', sparse=False)

scaled_numeric = pd.DataFrame(numeric_transformer.fit_transform(data_without_labels[numeric_features]), columns=numeric_features, index=data_without_labels.index)
encoded_categorical = pd.DataFrame(categorical_transformer.fit_transform(data_without_labels[categorical_features]), columns=categorical_transformer.get_feature_names_out(), index=data_without_labels.index)
binary_df = data_without_labels[binary_features]  # Assuming binary features don't need transformation
'''

# selects only numerical data
numeric_features = merged_df.select_dtypes(include=['float64', 'int64']).columns
merged_df = merged_df[numeric_features]
mortality_label = merged_df[['mortality_label']]
merged_df = merged_df.drop(columns=['SUBJECT_ID', 'mortality_label'])

# Scales data
cols = list(merged_df.columns)
for i in range(len(cols)):
    cols[i] = "".join([i for i in cols[i] if i not in ['[', ']', ',', '>', '<']]).replace(" ", '_')
merged_df.columns = cols
numeric_df = pd.concat((mortality_label, merged_df), axis = 1)

In [ ]:
try:
    numeric_df.drop(columns = ['SUBJECT_ID'], inplace = True)
except:
    pass
    
features = numeric_df.copy().drop(columns = ['mortality_label'])
features = features.dropna(axis=1, how='any')
numpy_features = np.array([[item for item in row] for row in features.to_numpy()])
numeric_transformer = StandardScaler()
numpy_features = numeric_transformer.fit_transform(numpy_features)

#imputer = MICE()
#print('Imputing...')
#unlabelled_data = imputer.fit_transform(numpy_features)

top_features = analyze_features(features_scaled=numpy_features, 
                 feature_names=features.columns,
                 y=list(numeric_df['mortality_label']), 
                 top_n_features=10, 
                 return_top_features=True)

In [9]:
# Function for equalizing data
def equalize_data(data, labels):
    data["Death"] = labels
    data = data.sample(frac = 1) 
    true_data = data[data["Death"] == 1]
    false_data = data[data["Death"] == 0]
    true_length, false_length = len(true_data), len(false_data)
    
    # Equalizes labels
    if true_length > false_length:
        sample_fraction = false_length / true_length
        true_data = true_data.sample(frac = sample_fraction)
    else:
        sample_fraction = true_length / false_length
        false_data = false_data.sample(frac = sample_fraction)
        
    # Recombines data
    data = pd.concat((true_data, false_data), axis = 0)
    labels = list(data['Death'])
    features = data.drop(columns = ['Death'])
    return features, labels


def feature_analysis(features, labels, top_features = None):
    # Analyzes feature importance via a variety of methods and filters features if enabled
    if top_features:
        feature_names = analyze_features(features_scaled = features.to_numpy(), feature_names = list(features.columns), y = labels,
                                    return_top_features = True, top_n_features = top_features,)
        return feature_names
    else:
        analyze_features(features_scaled = features.to_numpy(), feature_names = list(features.columns), y = labels, return_top_features = False)
    
# Tests all models on the aggregated features and data
def test_models(features, # Feature df = HAVE IT FILTERED  
                labels, # Mortality labels
                feature_names = None, # Feature names 
                equalize = True,
                rescale = True): # Equalize labels  

    # Initializes scaler
    scaler = StandardScaler()
    if feature_names != None:
        features = features[feature_names]
        
    print(f'Number of features: {len(features.columns)}')
    
    def k_fold_validation_for_sklearn(clf, features, labels, name='Random Forest'):
        print(f'Starting analysis for {name}')
        aucs = []
        mccs = []  # List to store MCC scores
        mean_fpr = np.linspace(0, 1, 100)
    
        for i, (train_index, test_index) in tqdm.tqdm(enumerate(kf.split(features)), total=n_splits):
            
            X_train, X_test = features[train_index], features[test_index]
            y_train, y_test = labels[train_index], labels[test_index]
            
            # Scale features if required
            if rescale == True:
                X_train = scaler.fit_transform(X_train)
                X_test = scaler.transform(X_test)
            
            # Fit and predict
            clf.fit(X_train, y_train)
            y_prob = clf.predict_proba(X_test)[:, 1]
            y_pred = clf.predict(X_test)  # Predictions for MCC
    
            # Compute ROC curve and AUC for this fold
            fpr, tpr, _ = roc_curve(y_test, y_prob)
            roc_auc = skauc(fpr, tpr)
            aucs.append(roc_auc)
            
            # Compute MCC for this fold
            mcc = matthews_corrcoef(y_test, y_pred)
            mccs.append(mcc)
    
            # Interpolate all ROC curves at this points
            interp_tpr = np.interp(mean_fpr, fpr, tpr)
            interp_tpr[0] = 0.0
            plt.plot(mean_fpr, interp_tpr, alpha=0.4, label=f'Split {i + 1} AUC = {round(roc_auc, 3)}')
    
        plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
    
        # Calculate the mean AUC, std deviation, mean MCC
        mean_auc = np.mean(aucs)
        std_auc = np.std(aucs)
        mean_mcc = np.mean(mccs)  # Calculate mean MCC
        print(f'Mean AUC: {mean_auc:.3f} ± {std_auc:.3f}')
        print(f'Mean MCC: {mean_mcc:.3f}')
        plt.title(f'{name}, ROC (AUC = {mean_auc:.3f} ± {std_auc:.3f})')
        plt.legend(loc="lower right")
        plt.show()
    
        return clf
    
    # Equalizes labels if enabled
    if equalize == True:
        features, labels = equalize_data(features, labels) 

    print(f'Number of processed data samples: {len(labels)}')
    
    # Prepare the data
    features_array, labels = features.to_numpy(), np.array(labels)
    
    # Random Forest Classifier
    rf_clf = RandomForestClassifier(n_estimators=1000, random_state=42)
    rf_clf = k_fold_validation_for_sklearn(rf_clf, features_array, labels)
    
    # XGBoost Classifier
    xgb_clf = XGBClassifier(n_estimators=1000, learning_rate=0.05, use_label_encoder=False, eval_metric='logloss',
                           name = 'XGBoost')
    xgb_clf = k_fold_validation_for_sklearn(xgb_clf, features_array, labels, name = 'XGBoost')
    
    # Custom-built network
    class NeuralNet(plit.LightningModule):
        def __init__(self, input_size, hidden_size=32):
            super(NeuralNet, self).__init__()
            
            self.network = nn.Sequential(
                nn.Linear(input_size, hidden_size * 8), nn.ReLU(),
                nn.BatchNorm1d(hidden_size * 8), nn.Dropout(0.3),
                
                nn.Linear(hidden_size * 8, hidden_size * 4), nn.ReLU(),
                nn.BatchNorm1d(hidden_size * 4), nn.Dropout(0.3),
                
                nn.Linear(hidden_size * 4, hidden_size * 2), nn.ReLU(),
                nn.BatchNorm1d(hidden_size * 2), nn.Dropout(0.3),
                
                nn.Linear(hidden_size * 2, hidden_size), nn.ReLU(),
                
                nn.Linear(hidden_size, hidden_size), nn.ReLU(),
                nn.BatchNorm1d(hidden_size), nn.Dropout(0.5),
            )

            self.final_linear = nn.Linear(hidden_size, 1)
            self.sigmoid = nn.Sigmoid()
            self.residual_transform = nn.Linear(input_size, hidden_size)
            
            # L1 and L2 regularization are defined but not used as layers
            
        def forward(self, x):
            out = self.network(x)  # Pass input through the network layers
            residual = self.residual_transform(x)  # Apply residual transformation to the original input
            out += residual  # Add residual connection
            out = self.final_linear(out)
            out = self.sigmoid(out)
            return out.squeeze()

        def training_step(self, batch, batch_idx):
            inputs, labels = batch
            outputs = self(inputs)
            loss = nn.BCELoss()(outputs, labels.view(-1))  # BCELoss for binary classification

            # Apply L1 and L2 regularization
            l1_reg = torch.tensor(0.)
            l2_reg = torch.tensor(0.)
            for param in self.parameters():
                l1_reg = l1_reg + torch.norm(param, 1)
                l2_reg = l2_reg + torch.norm(param, 2)

            # Regularization strengths need to be defined; for example:
            lambda1, lambda2 = 0.005, 0.005
            loss += lambda1 * l1_reg + lambda2 * l2_reg
            self.log("train_loss", loss)
            return loss

        def configure_optimizers(self):
            optimizer = torch.optim.AdamW(self.parameters(), lr=0.001)
            return optimizer
    
    def k_fold_validation_for_pytorch(model_class, features, labels, input_size):
        aucs = []
        mean_fpr = np.linspace(0, 1, 100)
        plt.figure()
        for train_index, test_index in tqdm.tqdm(kf.split(features), total = n_splits,):
            # Prepare data
            X_train, X_test = features[train_index], features[test_index]
            y_train, y_test = labels[train_index], labels[test_index]
           
            # Convert to tensors
            X_train_tensor, y_train_tensor = torch.FloatTensor(X_train), torch.FloatTensor(y_train)
            X_test_tensor, y_test_tensor = torch.FloatTensor(X_test), torch.FloatTensor(y_test)
            
            # DataLoader setup
            train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
            train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
            
            # Model setup
            model = model_class(input_size=input_size)
            trainer = plit.Trainer(max_epochs=20)
        
            trainer.fit(model, train_loader)
            
            # Evaluate
            model.eval()
            with torch.no_grad():
                y_prob = model(X_test_tensor).numpy()
            
            # Compute ROC curve and AUC for this fold
            fpr, tpr, _ = roc_curve(y_test, y_prob)
            roc_auc = skauc(fpr, tpr)
            aucs.append(roc_auc)
            
            interp_tpr = np.interp(mean_fpr, fpr, tpr)
            interp_tpr[0] = 0.0
            plt.plot(mean_fpr, interp_tpr, alpha=0.4)
        
        plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        mean_auc = np.mean(aucs)
        std_auc = np.std(aucs)
        plt.title(f'Custom Neural Network Mean ROC (AUC = {mean_auc:.3f} $\pm$ {std_auc:.3f})')
        plt.legend(loc="lower right")
        plt.show()

    return rf_clf, xgb_clf, scaler
    #print("Custom Neural Network")
    #k_fold_validation_for_pytorch(NeuralNet, features_array, labels, input_size=features.shape[1])


In [ ]:
# Tests models
test_models(features, 
                numeric_df['mortality_label'], # Mortality labels
                feature_names = top_features, # Feature names 
                equalize = True,
                rescale = True) # Equalize labels  